# Transform Messages for Non-OpenAI Models

There are a large variety of models available beyond OpenAI's and they all have different capabilities. Smaller context windows and different API's can require tweaks to your workflow in order to work with them.

If you're new to Transform Messages, see the [introduction to Transform Messages](/docs/topics/handling_long_contexts/intro_to_transform_messages).

## Reducing context sizes

Although context windows are increasing, there are still a large number of models that have context windows (e.g. 2K, 4K, or 8K tokens) which may be overwhelmed by your workflow's messages.

To handle longer contexts using transforms, essentially reducing them effectively for smaller context windows, please see the page on [compressing text](/docs/topics/handling_long_contexts/compressing_text_w_llmligua).

## Incorporating an agent's name

Interestingly, the agent's name, such as Jack in the below example, is not included in messages when using non-OpenAI models. This means that there is no way of the name being known by the model during inference, unless we include it in the body of the message text.

```python
comedian = ConversableAgent(
    name="Jack", # Not included in messages for non-OpenAI inference
    llm_config=phi2,
    system_message="Your name is Jack and you are a comedian.",
)
```

When using OpenAI models, the name field is included and examples in the AutoGen documentation may rely on this fact. Therefore, it may not be an issue in your existing workflows, however it's important to be aware of and be able to cater for it.

In the simple two-agent chat example, below, we will use a `TextMessageContentName` transform, available from the Transforms module, to add in the name of the agents to the messages.

As we won't be using OpenAI, we will use the Anthropic client to demonstrate.

We'll start by importing our modules and setting our config.

In [1]:
import os

from autogen import ConversableAgent

config_list_claude = [
    {
        "model": "claude-3-5-sonnet-20240620",
        "api_key": os.getenv("ANTHROPIC_API_KEY"),
        "api_type": "anthropic",
        "cache_seed": None,  # Ensure we're not caching any results
    }
]

Now we add two agents, both comedians who will make up jokes about the other using their name.

In [2]:
cathy = ConversableAgent(
    "Cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    "Joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",
)

Let's start the chat without using the transform and see how it performs.

In [3]:
result = joe.initiate_chat(cathy, message="People say I have a funny name, tell me a joke about it.", max_turns=2)

As we can see from the chat, Cathy doesn't know Joe's name at the start.

Now, we'll create a transform that injects the names into the messages and apply that transform to both agents.

In [4]:
# Import our transforms
from autogen.agentchat.contrib.capabilities import transform_messages
from autogen.agentchat.contrib.capabilities.transforms import TextMessageContentName

# Create a name transform
# This will inject the agent's name for a message into the start of the message content.
# E.g. "'Jack' said\n..."
name_transform = TextMessageContentName(position="start", format_string="'{name}' said:\n")

# Create the TransformMessages that will be applied.
# In this case we are only putting in one transform but you could
# stack the transforms if you also wanted to do others, like
# compress the text. Transforms are performed sequentially.
context_handling = transform_messages.TransformMessages(transforms=[name_transform])

# Add it to both agents so when they run inference it will apply to the messages
context_handling.add_to_agent(cathy)
context_handling.add_to_agent(joe)

Let's try that chat again now that we're injecting the names.

In [5]:
result = joe.initiate_chat(cathy, message="People say I have a funny name, tell me a joke about it.", max_turns=2)

We can see from this conversation that Cathy uses Joe's name in her first response message, showing that incorporating the name using a transform has enabled the Cathy agent to *recognise* Joe.

Where the transform used above becomes essential is in a Group Chat using the `auto` selection mode (default), when the Group Chat Manager is selecting an agent based on their `name`.

## Transforms in group chats

As noted above, it is important when using non-OpenAI models to inject the agent's name into the messages when you are using `auto` agent selection mode. By doing so, you are giving the model the best chance at understanding which agent belongs to each message.

Additionally, group chats can involve a large number of messages and, therefore, tokens. So, to assist with keeping the context used within your model's context window you can use a compression transform.

Below is a group chat example that incorporates these two transforms and relies on the LLM using agent names to determine and select the next agent.

We'll use Anthropic's Claude 3.5 Sonnet config from the previous example as the LLM for all agents and the group chat manager (which selects the next agent to speak).

The scenario in the example is the production of two kid-friendly long-form articles on cloud formations.

Let's start by creating our three team members:

- **Subject_Expert** will select a cloud formation and provide some bullet points about it.
- **Writer** will write long-form content, about 2,000 words, for the selected cloud formation based on the bullet points.
- **Scheduler** is responsible for delivering the task to the group chat manager, determining if we need to continue writing more articles, and to terminate the group chat by saying 'TERMINATE'.

In [11]:
sme_agent = ConversableAgent(
    "Subject_Expert",
    system_message="You're a subject matter expert on cloud formations and work in a team with a scheduler and a writer. Every time you're asked to speak it's for a new article. You must prepare for a new article by selecting a the cloud formation, providing a summary of that formation and the impact on weather, in bullet points. Make it kid friendly. Aim for a dozen bullet points. Your task is only to provide topics and bullet points on new articles, don't review any previously written articles.",
    description="An expert on cloud formations, great at developing ideas to write about.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",
)

scheduler = ConversableAgent(
    "Scheduler",
    system_message="You're a marketing expert, managing the production of a specific number of articles. Count the number of articles written and once they have been written say the word 'TERMINATE'.",
    description="A marketing expert that's excellent at managing the production of articles.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",
)

writer = ConversableAgent(
    "Writer",
    system_message="You're a writer of online news articles on scientific topics, written for an audience of primary school students. Aim for 2,000 words for each article.",
    description="An excellent writer, takes given topics and writes long-form articles.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",
)

Now we create our two transforms, one for injecting the `name` and the other to compress the messages if the estimated token count for all messages combined is greater than 1,000 tokens.

As these transforms will be applied to the nested chat in a group chat where the next speaker is selected, we add a filter to the transforms to not apply to `system` messages and to messages from the `checking_agent` who is the agent within the nested chat for selecting the next speaker.

These exclusions are used to minimise any loss of instruction in those messages as they are critical for speaker selection.

In [12]:
from autogen.agentchat.contrib.capabilities.text_compressors import LLMLingua
from autogen.agentchat.contrib.capabilities.transforms import TextMessageCompressor, TextMessageContentName

# Create transform to inject name
# This will inject the agent's name for a message into the start of the message content.
# E.g. "'Subject_Expert' said\n..."
name_transform = TextMessageContentName(
    position="start",
    format_string="'{name}' said:\n",
    filter_dict={
        "role": ["system"],
        "name": ["checking_agent"],
    },  # don't add the name for the select speaker-specific nested-chat agents
)

# Create transform to compress messages
# If you don't have LLMLingua installed: pip install LLMLingua
llm_lingua = LLMLingua()
compress_transform = TextMessageCompressor(
    text_compressor=llm_lingua,
    min_tokens=1000,  # Don't compress if total tokens in list of messages is <= 1000
    filter_dict={
        "role": ["system"],
        "name": ["checking_agent"],
    },  # don't compress messages specifically for the select speaker prompts
)

# Create the TransformMessages that will be applied
# In this case we are only putting in one transform but you could
# stack the transforms if you also wanted to do others, like
# compressing the text. Transforms are performed sequentially.
select_speaker_transforms = transform_messages.TransformMessages(
    transforms=[
        compress_transform,
        name_transform,
    ]
)

With transforms created, we can apply them to the group chat's select speaker nested chat.

In addition to the application of the transforms to the group chat's `select_speaker_transform_messages` parameter, we are providing explicit instructions on the order of agents within the `select_speaker_message_template`.

In [13]:
from autogen import GroupChat, GroupChatManager

group_chat = GroupChat(
    agents=[sme_agent, scheduler, writer],
    messages=[],
    max_round=10,
    select_speaker_message_template="""You manage a team that produces and releases articles.
        The roles available in the team are:
        {roles}
        Take the task given and coordinate the production of one or more articles.
        The order for each article should be the Subject_Expert first, then the Writer to write an article, then the Scheduler to review and determine if more are required.
        Finally, you can output the word 'TERMINATE' to signify the end of the task.""",
    select_speaker_prompt_template="Read the above conversation, select the next person from {agentlist} and only return the role.",
    # Transforms applied to the group chat speaker selection when in 'auto' mode
    select_speaker_transform_messages=select_speaker_transforms,
    select_speaker_auto_verbose=True,  # See the selection process
)

manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": config_list_claude},
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
)

We will also apply the transforms to each of the agents so that when they are getting the messages to respond to they are compressed and have the names of the agents injected.

In [14]:
# We add the transforms to the team of agents so they understand who has said what and the messages are compressed to save tokens
select_speaker_transforms.add_to_agent(sme_agent)
select_speaker_transforms.add_to_agent(scheduler)
select_speaker_transforms.add_to_agent(writer)

Start the chat and show the cost at the end.

Note: `select_speaker_auto_verbose` was set to True on the group chat so you can see the speaker selection process in between each message.

In [15]:
chat_result = scheduler.initiate_chat(
    recipient=manager,
    message="We need a couple of articles on different cloud formations, let's get some help on creating them!",
)

print(f"The cost of the chat was:\n{chat_result.cost}")

There's a lot to digest in the above code, let's break it down.

1. It successfully did the task, getting two articles written (Yay!)
2. The sequence of agents was selected successfully: Scheduler to Subject_Expert to Writer then back to Scheduler to Subject_Expert to Writer to Scheduler who then terminates
3. Transform messages show the number of messages incorporating the name and the tokens saved during the process
4. We occasionally see the next agent name "Please continue." being proposed by the LLM and this is because these continuation messages are inter-woven in the messages sent to Anthropic's API. This is handled well with the select speaker retries, but further prompt tuning could help eliminate these anomalies

Additional notes:

- Would this have worked without the transforms? Taking out the transforms resulted in a run producing the following incorrect sequence: Scheduler to Subject_Export to Writer back to Writer then to Scheduler.
- Tweaking - tweaking the system messages and descriptions for agents and the group chat select speaker nested chat also played a large role in steering the LLM to the correct output. A combination of prompt engineering and the transforms may be required to achieve consistent results.

